In [1]:
import win32com.client
import datetime
import time
import psutil
import logging
from playsound import playsound

In [2]:
# ----- Configuration Section -----
CHECK_INTERVAL = 30         # Interval (in seconds) between each check.
ALERT_BEFORE_MINUTES = 2      # Trigger alert if a meeting starts within next 2 minutes.
RINGTONE_FILE = "ring1.wav"  # Path to your custom ringtone sound file.
# ----------------------------------

In [3]:

# Set up logging for output details.
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [4]:
def is_teams_running():
    """
    Check if Microsoft Teams is running by scanning the processes.
    Returns True if the Teams process is found.
    """
    for proc in psutil.process_iter(['name']):
        try:
            if 'Teams.exe' in proc.info['name']:
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    return False

In [5]:
def get_upcoming_meetings():
    """
    Reads the Outlook calendar for the current day and finds all meetings
    that will start within the next ALERT_BEFORE_MINUTES.
    Returns a list of meeting appointment objects.
    """
    upcoming_meetings = []
    try:
        # Connect to Outlook via COM.
        outlook = win32com.client.Dispatch("Outlook.Application")
        namespace = outlook.GetNamespace("MAPI")
        calendar = namespace.GetDefaultFolder(9)  # 9 = Calendar folder in Outlook.
        
        # Define the time range for today's appointments.
        today = datetime.date.today()
        start = datetime.datetime.combine(today, datetime.time.min)
        end = datetime.datetime.combine(today, datetime.time.max)
        
        # Get items, include recurring events, and sort by start time.
        items = calendar.Items
        items.IncludeRecurrences = True
        items.Sort("[Start]")
        
        # Create a restriction to only get appointments today.
        restriction = "[Start] >= '" + start.strftime("%m/%d/%Y %H:%M %p") + "' AND [End] <= '" + end.strftime("%m/%d/%Y %H:%M %p") + "'"
        restricted_items = items.Restrict(restriction)
        
        now = datetime.datetime.now()
        for appointment in restricted_items:
            meeting_start = appointment.Start
            # If meeting_start is a string, convert it (usually it’s a datetime)
            if isinstance(meeting_start, str):
                meeting_start = datetime.datetime.strptime(meeting_start, "%m/%d/%Y %H:%M %p")
            
            # Check if the meeting will start soon.
            if 0 <= (meeting_start - now).total_seconds() <= ALERT_BEFORE_MINUTES * 60:
                upcoming_meetings.append(appointment)
    except Exception as e:
        logging.error("Error reading Outlook calendar: " + str(e))
    return upcoming_meetings

In [6]:
def main_loop():
    """
    Main loop that continuously checks for upcoming meetings.
    Plays the ringtone if a meeting is about to start and Teams is running.
    """
    notified = set()  # To avoid duplicate notifications for the same meeting.
    logging.info("Starting Teams Meeting Notification Agent...")
    while True:
        now = datetime.datetime.now()
        upcoming = get_upcoming_meetings()
        if upcoming:
            logging.info(f"Found {len(upcoming)} meeting(s) starting soon.")
            if is_teams_running():
                for meeting in upcoming:
                    meeting_id = meeting.EntryID  # Unique identifier for each meeting.
                    if meeting_id not in notified:
                        logging.info(f"Triggering alert for meeting: '{meeting.Subject}' scheduled at {meeting.Start}")
                        try:
                            playsound(RINGTONE_FILE)
                            notified.add(meeting_id)
                        except Exception as e:
                            logging.error("Error playing sound: " + str(e))
            else:
                logging.info("Teams is not running; skipping notification.")
        else:
            logging.debug("No upcoming meetings detected.")
        time.sleep(CHECK_INTERVAL)

In [7]:
if __name__ == "__main__":
    main_loop()


2025-04-10 15:30:39,254 - INFO - Starting Teams Meeting Notification Agent...


KeyboardInterrupt: 

In [1]:
from dateutil import tz

local_tz = tz.tzlocal()

def to_local(dt):
    if dt.tzinfo is None:
        return dt.replace(tzinfo=tz.tzutc()).astimezone(local_tz)
    else:
        return dt.astimezone(local_tz)


In [4]:
start = to_local(now)


NameError: name 'now' is not defined

In [3]:

from playsound import playsound

In [6]:
playsound("ring1.wav")